In [1]:
import sys
sys.path.append('..')

In [2]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP

import os

model = LlamaCPP(
    model_path=os.path.join("/mnt/d/wsl/llamacpp/models", "neural-chat-7b-v3-3.Q5_K_M.gguf"),
    temperature=0.1,
    max_new_tokens=512,
    # callback_manager=callback_manager,
    # verbose=True, # Verbose is required to pass to the callback manager
    # echo=True,
    context_window=1024*32,
    model_kwargs={"n_gpu_layers": -1}
)


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /mnt/d/wsl/llamacpp/models/neural-chat-7b-v3-3.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = intel_neural-chat-7b-v3-3
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:

In [5]:
import fnmatch

def find_files_with_extension(root_dir, extension):
    
    file_list = []
    for root, _, files in os.walk(root_dir):
        for filename in fnmatch.filter(files, f'*.{extension}'):
            file_list.append(os.path.join(root, filename))
    return file_list

documents = SimpleDirectoryReader(input_files=find_files_with_extension("srd", "json")).load_data()

embed_model="local:BAAI/bge-small-en-v1.5"

index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

ImportError: `llama-index-embeddings-huggingface` package not found, please run `pip install llama-index-embeddings-huggingface`